In [1]:
from ase.io import read
import time
import sys
sys.path.append('../code')
from Generate_Descriptors import get_dscribe_descriptors

In [33]:
C7O2H10_structs = read("../data/C7O2H10.extxyz.gz", ":2000")

In [3]:
graphenes = read("../data/ManyGraphenes.extxyz.gz")

In [12]:
def TickTock(func, *args, **kwargs):
    tick = time.time()
    func_output = func(*args, **kwargs)
    tock = time.time()
    return func_output, tock - tick

In [34]:
species = ["C", "O", "H", "N", "Ca", "Li"]
rcut = 3
nmax = 12
lmax = 9
SoapList, SoapTime = TickTock(get_dscribe_descriptors, C7O2H10_structs, species = species, rcut = rcut, nmax = nmax, lmax = lmax, is_global=False, return_derivatives = False)
print(SoapTime)

23.963632822036743


In [41]:
species = ["C", "O", "H", "N", "Ca", "Li"]
rcut = 3
nmax = 12
lmax = 9
SoapList, SoapTime = TickTock(get_dscribe_descriptors, graphenes, species = species, rcut = rcut, nmax = nmax, lmax = lmax, is_global=False, return_derivatives = False)
print(SoapTime)

0.09846997261047363


In [44]:
from ase.build import bulk
diamond = bulk("C", 'diamond')
diamond
Structure3DPlot(diamond).Plot()

Atoms(symbols='C2', pbc=True, cell=[[0.0, 1.785, 1.785], [1.785, 0.0, 1.785], [1.785, 1.785, 0.0]])

In [ ]:
# Breakdown of the derivatives output tensor from soap.derivatives(). This explains what the different indices mean in different situations

use_atom_centers = True
use_local_descriptors = True
n_structures = 5

averaging_keyword = "off" if use_local_descriptors else "outer"
my_water_molecule = ase.build.molecule('H2O')
if n_structures == 1:
    my_atoms_list = my_water_molecule
    print("This breakdown of the derivatives tensor is for a single (1) water molecule.")
elif n_structures > 1:
    my_atoms_list = [my_water_molecule] * n_structures
    print("This breakdown of the derivatives tensor is for {} water molecules.".format(n_structures))
    print("To access the tensor values without defining a tensor for each molecule, add an extra index in front. I didn't do this here because it was easier not to.")


soap = SOAP(average=averaging_keyword,  species=["H", "O"], periodic=False, rcut=1.2, nmax=1, lmax=1)

if use_atom_centers:
    positions = my_water_molecule.get_positions()
else:
    positions = [[0, 0, 0],[5,2,3]]

if n_structures == 1:
    positions_list = positions
elif n_structures > 1:
    positions_list = [positions]*len(my_atoms_list)

my_soap = soap.create(my_atoms_list)
derivatives, descriptors = soap.derivatives(my_atoms_list, positions=positions_list , method="auto")

print("")

for k in range(n_structures):
    if n_structures == 1:
        derivative_k = derivatives
        print("For the water molecule soap.derivatives() output, D (soap derivatives) and S (soaps):")
    elif n_structures > 1:
        derivative_k = derivatives[k]
        print("For water molecule #{} soap.derivatives() output, D (soap derivatives) and S (soaps):".format(k))
    for i in range(len(derivative_k)):
        print("---------------------------------------------------------------------------------------------")
        print("---------------------------------------------------------------------------------------------")
        if use_local_descriptors:
            atom_i = "atom " + my_water_molecule[i].symbol + str(my_water_molecule[i].index) + " " if use_atom_centers else ""
            print("D[{}] = derivatives of S[{}], a local soap descriptor centered at {}({:5.2f}, {:5.2f}, {:5.2f})".format(i, i, atom_i, *positions[i]))
        else:
            print("D[{}] = derivative of S[{}], the global soap descriptor averaged over {} centers".format(i, i, len(positions[i])))
        print("---------------------------------------------------------------------------------------------")
        print("---------------------------------------------------------------------------------------------")
        for j in range(len(my_water_molecule)):
            atom_j = my_water_molecule[j].symbol + str(my_water_molecule[j].index)
            print("D[{}, {}] = derivative of S[{}] with respect to atom {} positions".format(i, j, i, atom_j))
            print("--------------------------------------------------------------")

            print("D[{}, {}, {}] = dS[{}]/dX{}: ".format(i, j, 0, i, j), "[" + "  ".join(["{:7.4f}".format(p) for p in derivative_k[i,j,0]]) + "]")
            print("D[{}, {}, {}] = dS[{}]/dY{}: ".format(i, j, 1, i,j), "[" + "  ".join(["{:7.4f}".format(p) for p in derivative_k[i,j,1]]) + "]")
            print("D[{}, {}, {}] = dS[{}]/dZ{}: ".format(i, j, 2, i,j), "[" + "  ".join(["{:7.4f}".format(p) for p in derivative_k[i,j,2]]) + "]")
            #print("")
            print("--------------------------------------------------------------\n")



Fiddling with the code from https://github.com/SINGROUP/dscribe/issues/63 to explore the new 'attach' flag

This was in the main miniGAP notebook and I moved it to make that more usable

The dscribe author made an interesting point on this issue page about why there are 0s in the derivative output (l=0)

In [1]:
import numpy as np
from ase import Atoms
from ase.vibrations import Vibrations
from dscribe.descriptors import SOAP

atoms = Atoms(symbols='HF', positions=[[0, 0, 0], [0, 0, .9382]], pbc=False)
soap = SOAP(rcut = 6.0, nmax = 1, lmax = 1, species = atoms.get_chemical_symbols(), dtype="float64" )

print('\nSOAP.create:')
soaps = soap.create(atoms)  # 6 features in total

for i in range(len(soaps)):
    print("[ ", end="")
    for j in range(len(soaps[i])):
        print(" {:6.2f} ".format(soaps[i,j]), end="")
    print(" ]")

# DSCRIBE derivatives
atom_coordinates = [[0,0,0], [0,0,.9382]]# + np.ones(atoms.positions.shape)*1e-6#atoms.positions + np.ones(atoms.positions.shape)*1e-8
dscribe_derivs = soap.derivatives(atoms, method='numerical', return_descriptor=False, attach=True, positions=atom_coordinates)
nonzero_min =np.min(np.abs(np.ma.masked_equal(dscribe_derivs, 0, copy=True)))
print('\nSOAP.derivatives:')
print(nonzero_min)
#PrintNoScientificNotation(dscribe_derivs)
n_centers, n_atoms, _, n_features = dscribe_derivs.shape
dscribe_derivs = np.moveaxis(dscribe_derivs, 0, 2).reshape(( 3 * n_atoms, n_atoms * n_features)).T

#PrintNoScientificNotation(dscribe_derivs)
for i in range(len(dscribe_derivs)):
    print("[ ", end="")
    for j in range(len(dscribe_derivs[i])):
        print(" {:6.2f} ".format(dscribe_derivs[i,j]), end="")
    print(" ]")


# Numerical derivatives using finite displacements
eps = 1e-5  # finite difference
vib = Vibrations(atoms, delta=eps, nfree=2)
atoms_displacements = list(vib.iterimages())  # create finite displacements

s = soap.create(atoms_displacements)
s -= s[0]  # subtract soap for unperturbed structure
s = s[1:]  # pop unperturbed structure
numerical_derivs = 0.5 * (s[1::2] - s[::2]) / eps  # centered finite difference scheme
numerical_derivs = numerical_derivs.reshape((3 * len(atoms), len(s[0].ravel()))).T
print('\nDerivatives by finite displacements:')
#print(numerical_derivs)
for i in range(len(numerical_derivs)):
    print("[ ", end="")
    for j in range(len(numerical_derivs[i])):
        print(" {:6.2f} ".format(numerical_derivs[i,j]), end="")
    print(" ]")


SOAP.create:
[    6.25    0.00    4.15    0.00    2.75    0.18  ]
[    2.75    0.18    4.15    0.00    6.25    0.00  ]

SOAP.derivatives:
0.06721823726990594
[    0.00    0.00    0.00    0.00    0.00    0.00  ]
[    0.00    0.00    0.00    0.00    0.00    0.00  ]
[    0.00    0.00    0.00    0.00    0.00   -3.63  ]
[    0.00    0.00    0.28    0.00    0.00    0.00  ]
[    0.00    0.00    0.00    0.00    0.00   -4.81  ]
[    0.00    0.00    0.00    0.00    0.00    0.07  ]
[    0.00    0.00    4.81    0.00    0.00    0.00  ]
[    0.00    0.00   -0.07    0.00    0.00    0.00  ]
[    0.00    0.00    3.63    0.00    0.00    0.00  ]
[    0.00    0.00    0.00    0.00    0.00   -0.28  ]
[    0.00    0.00    0.00    0.00    0.00    0.00  ]
[    0.00    0.00    0.00    0.00    0.00    0.00  ]

Derivatives by finite displacements:
[    0.00    0.00    0.00    0.00    0.00    0.00  ]
[    0.00    0.00    0.00    0.00    0.00    0.00  ]
[    0.00    0.00    3.63    0.00    0.00   -3.63  ]
[    0.0

In [2]:
atoms = Atoms(symbols='HF', positions=[[7.5, 7.5, 7.5], [7.5, 7.5, 8.4382]], pbc=False)
soap = SOAP(rcut = 6.0, nmax = 1, lmax = 0, species = atoms.get_chemical_symbols())

print('\nSOAP.create:')
print(soap.create(atoms))  # 6 features in total


# DSCRIBE derivatives
dscribe_derivs = soap.derivatives(atoms, method='auto', return_descriptor=False)
n_centers, n_atoms, _, n_features = dscribe_derivs.shape
dscribe_derivs = np.moveaxis(dscribe_derivs, 0, 2).reshape(( 3 * n_atoms, n_atoms * n_features)).T
print('\nSOAP.derivatives:')
print(dscribe_derivs)


SOAP.create:
[[6.25043443 4.14642847 2.75066786]
 [2.75066786 4.14642847 6.25043443]]

SOAP.derivatives:
[[ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -3.62760443]
 [ 0.          0.          0.          0.          0.         -4.81297819]
 [ 0.          0.          4.81297819  0.          0.          0.        ]
 [ 0.          0.          3.62760443  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]


In [3]:
positions=[[7.5, 7.5, 7.5], [7.5, 7.5, 8.4382]]
atoms = Atoms(symbols='HF', positions=positions, pbc=False)
soap = SOAP(rcut = 6.0, nmax = 1, lmax = 1, species = atoms.get_chemical_symbols(), dtype="float64")

print('\nSOAP.create:')
print(soap.create(atoms))  # 6 features in total

# DSCRIBE derivatives
dscribe_derivs = soap.derivatives(atoms, method='auto', return_descriptor=False)
n_centers, n_atoms, _, n_features = dscribe_derivs.shape
dscribe_derivs = np.moveaxis(dscribe_derivs, 0, 2).reshape(( 3 * n_atoms, n_atoms * n_features)).T
print('\nSOAP.derivatives:')
print(dscribe_derivs)


SOAP.create:
[[6.25043443 0.         4.14642847 0.         2.75066786 0.17596734]
 [2.75066786 0.17596734 4.14642847 0.         6.25043443 0.        ]]

SOAP.derivatives:
[[ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -3.62760443]
 [ 0.          0.          0.28273051  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -4.81297819]
 [ 0.          0.          0.          0.          0.          0.06721824]
 [ 0.          0.          4.81297819  0.          0.          0.        ]
 [ 0.          0.         -0.06721824  0.          0.          0.        ]
 [ 0.          0.          3.62760443  0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.28273051]
 [ 0.          0.          0.          0.          0.          0.        ]
 [ 